<a href="https://colab.research.google.com/github/XekoFrontend/google-colab-notebook/blob/main/rclone_gui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 **Install**


In [ ]:
# Cài đặt rClone
!sudo -v ; curl https://rclone.org/install.sh | sudo bash

In [ ]:
# Danh sách remotes đang có
!rclone listremotes

In [ ]:
# Khôi phục file config
!cp "/content/drive/MyDrive/Colab Notebooks/rclone.conf" /root/.config/rclone/rclone.conf

# ⚙️ **Config remotes**

Modify remotes

In [ ]:
# Tạo mới, chỉnh sửa, xóa hoặc kiểm tra remote.
!rclone config

Backup config

In [ ]:
# Backup config
!cp /root/.config/rclone/rclone.conf /content/rclone.conf
!rclone copy -v "/content/rclone.conf" "/content/drive/MyDrive/Colab Notebooks/backup/rclone-renamed.conf"

# 🎯 **Các lệnh Copy**

In [ ]:
# @title Basic: Copy files between Cloud Remotes

source_remote = "onedrive" # @param ["/content/drive/MyDrive/","/content/","googledrive1","onedrive","googledrive2","archive"]
input_folder = "" # @param {"type":"string","placeholder":"dùng `/` để phân cấp folder hoặc file, để trống là toàn bộ remote"}
# @markdown ###Lưu ý:
# @markdown - nếu thêm `/` ở cuối, chỉ copy nội dung bên trong
# @markdown - nếu không dùng `/` ở cuối thì copy cả folder
# @markdown ---
destination_remote = "archive" # @param ["archive","onedrive","googledrive2","googledrive1","/content/drive/MyDrive/","/content/"]
destination_folder = "" # @param {"type":"string","placeholder":"dùng `/` để phân cấp folder"}
# @markdown ---
# @markdown ### Hướng dẫn:
# @markdown - Copy bản mới (không dùng được trên archive): `--update`.
# @markdown - Bỏ qua các file đã tồn tại (**thường dùng trên archive.org**): `--ignore-existing`.
# @markdown - Chọn số luồng copy (nếu không dùng thì mặc định là 4)): `--transfers 8`.
# @markdown - Dùng cho S3 của Amazon (**archive.org dùng 1 luồng có vẻ nhanh**): `--s3-upload-concurrency 1`.

option_flag = "" # @param {"type":"string","placeholder":"thêm `--flag` nếu cần, dùng space phân cách các flag"}

# Kiểm tra và thực thi lệnh copy
if source_remote and destination_remote:
    # Chạy lệnh rclone với thông báo chi tiết
    !rclone copy -v "{option_flag}" "{source_remote}:{input_folder}" "{destination_remote}:{destination_folder}"
else:
    print("Lỗi: Vui lòng nhập cả Source Remote và Destination Remote.")

In [ ]:
# @title grok
# @markdown ### Hướng dẫn:
# @markdown - **Source Remote**: Chọn hoặc nhập remote nguồn (ví dụ: `haocenter`, `/content/drive/MyDrive/`). Nếu dùng thư mục cục bộ, thêm `/` ở cuối.
# @markdown - **Input Folder**: Nhập tên thư mục/file trong remote nguồn (ví dụ: `CBZ/`, để trống nếu muốn sao chép toàn bộ remote).
# @markdown - **Destination Remote**: Chọn remote đích (ví dụ: `archive`, `tsd06`). Nếu dùng thư mục cục bộ, thêm `/` ở cuối.
# @markdown - **Destination Folder**: Nhập tên thư mục đích (ví dụ: `Backup/`, để trống nếu muốn sao chép vào root remote).
# @markdown - **Transfer Option**:
# @markdown   - `Default (--update -v)`: Sao chép với cập nhật file mới hơn, chi tiết log.
# @markdown   - `Transfers (số luồng)`: Chọn số luồng (mặc định 8, có thể nhập số khác).
# @markdown   - `S3 Concurrency (1)`: Dùng 1 luồng cho upload S3 (phù hợp với Archive).
# @markdown - Nhấn **Run** để thực thi. Nếu sai cú pháp, sẽ hiển thị lỗi và hướng dẫn.

source_remote = "/content/drive/MyDrive/" # @param ["/content/drive/MyDrive/", "haocenter", "tsd06", "xuhao", "archive"]
input_folder = "" # @param {type:"string", allow-input: true}
destination_remote = "archive" # @param ["archive", "tsd06", "xuhao", "haocenter", "/content/drive/MyDrive/"]
destination_folder = "" # @param {type:"string", allow-input: true}
transfer_option = "S3 Concurrency (1)" # @param ["Default (--update -v)", "Transfers (số luồng)", "S3 Concurrency (1)"]
transfers_value = "" # @param {type:"string", allow-input: true}

# Xử lý tùy chọn chuyển dữ liệu
transfer_flags = ""
if transfer_option == "Default (--update -v)":
    transfer_flags = "--update -v"
elif transfer_option == "Transfers (số luồng)":
    try:
        transfers_value = int(transfers_value)
        if transfers_value > 0:
            transfer_flags = f"--transfers {transfers_value} -v"
        else:
            print("Lỗi: Số luồng phải lớn hơn 0. Mặc định dùng 8.")
            transfer_flags = "--transfers 8 -v"
    except ValueError:
        print("Lỗi: Vui lòng nhập số nguyên cho Transfers. Mặc định dùng 8.")
        transfer_flags = "--transfers 8 -v"
elif transfer_option == "S3 Concurrency (1)":
    transfer_flags = "--s3-upload-concurrency 1 -v"

# Xử lý đường dẫn
if source_remote and destination_remote:
    source_path = f"{source_remote}{input_folder}".rstrip("/") + "/" if input_folder else f"{source_remote}"
    dest_path = f"{destination_remote}{destination_folder}".rstrip("/") + "/" if destination_folder else f"{destination_remote}"

    # Thay đổi tùy chọn khi đích là archive
    if destination_remote == "archive":
        command_flags = transfer_flags.replace("--update", "--ignore-existing") if "--update" in transfer_flags else transfer_flags
    else:
        command_flags = transfer_flags

    # Chạy lệnh rclone
    !rclone copy {command_flags} "{source_path}" "{dest_path}"
else:
    print("Lỗi: Vui lòng nhập cả Source Remote và Destination Remote. Xem hướng dẫn ở trên!")

# 👓 **Xem Nội Dung Remote**

In [ ]:
# @markdown ### Hướng dẫn:
# @markdown - **List Mode**: Chọn cách hiển thị nội dung:
# @markdown   - `ls`: Liệt kê file và kích thước (mặc định).
# @markdown   - `lsf`: Chỉ hiển thị tên file/thư mục.
# @markdown   - `lsd`: Chỉ hiển thị tên thư mục con.
# @markdown   - `size`: Hiển thị tổng kích thước của thư mục.

list_remote = "/content/drive/MyDrive/" # @param ["/content/drive/MyDrive/", "haocenter","tsd06","xuhao","archive"]
input_folder = "tinhhe-lontien-thyetmin/" # @param {"type":"string"}
list_mode = "lsf" # @param ["lsf","lsd","ls","size"]
# @markdown 🔎 Lọc file:
file_type = "mkv" # @param {"type":"string"}
file_type_sum = "Default" # @param ["Default", "Sum"]

# Xử lý file_type_sum thành lệnh phù hợp
sum_command = "" if file_type_sum == "Default" else " | wc -l"


!rclone {list_mode} "{list_remote}:{input_folder}"  | grep ".{file_type}"  {sum_command}


# ======================================
# grep ".mp4": Lọc các dòng có chứa .mp4.
# wc -l: Đếm số dòng (tức là số file .mp4).

# 🗃️ **.Rar .Zip .7zip**

## Rar

In [ ]:
# Install unrar
!apt-get install unrar

In [ ]:
# Giải nén với password (ex: DDDTT)
!unrar x -pdomdom2010 /content/copy_temp/*.zip /content/copy_temp/extracted/

## Zip

In [ ]:
# Install unzip
!apt-get install -y unzip

In [ ]:
# Giải nén file .zip với password (nếu có)
!unzip -P kirakira /content/copy_temp/Ed01.zip -d /content/copy_temp/extracted/

In [ ]:
# Dùng vòng lặp `for` Giải nén nhiều file .zip trong thư mục với password (nếu có)
import os
for zip_file in os.listdir("/content/copy_temp/2/"):
    if zip_file.endswith(".zip"):
        !unzip -P domdom2010 "/content/copy_temp/2/{zip_file}" -d /content/copy_temp/2/s2/
        print(f"Đã giải nén {zip_file}")

---
### uncheck

rar

In [ ]:
# hình như là nén files, hoặc ghép files. cần thử lại.
!cat /content/torrent_download/HI1\ DDDTT.part*.rar > /content/HI1_DDDTT_full.rar

In [ ]:
# Split file nén
!split -b 10G /content/HI1_DDDTT_full.rar /content/HI1_DDDTT_part_
# copy file trùng tên, chhia theo thứ tự.
!rclone copy /content/HI1_DDDTT_part_* archive:tinhhe-lontien-thyetmin/1990/ -v

7zip
* p7zip-full hỗ trợ hầu hết định dạng (.7z, .zip, .rar, .iso, .tar, .gzip, v.v.), nhưng hiệu suất giải nén .rar tốt hơn với unrar.

In [ ]:
!apt-get update
!apt-get install -y p7zip-full

In [ ]:
!7z x /content/file.7z -o/content/extracted/ -pYOUR_PASSWORD

In [ ]:
!7z x /content/file.iso -o/content/extracted/

In [ ]:
import os

# Đường dẫn thư mục chứa file nén
source_folder = "/content/copy_temp/2/"
extract_folder = "/content/copy_temp/2/zip2/"
password = "domdom2010"  # Thay bằng password thực tế

# Tạo thư mục đích nếu chưa có
!mkdir -p {extract_folder}

# Lặp qua các file trong thư mục
for filename in os.listdir(source_folder):
    if filename.endswith(".zip"):
        file_path = os.path.join(source_folder, filename)
        # Sử dụng 7z để giải nén
        !7z x "{file_path}" -o"{extract_folder}" -p{password} -y
        if os.path.exists(os.path.join(extract_folder, filename.replace(".zip", ""))):
            print(f"Đã giải nén {filename} thành công")
        else:
            print(f"Lỗi: Không thể giải nén {filename}. Kiểm tra file hoặc password!")

cai đặt cả 3

In [ ]:
# Cài đặt các công cụ giải nén
!apt-get update
!apt-get install -y unrar unzip p7zip-full

# 📺 **yt-dlp**

## ⚙️ Install yt-dlp & ffmpeg
* ⚠️Chú ý dùng ! trước command
* Nếu lỗi 403 Forbidden, thêm --geo-bypass

In [ ]:
pip install -U yt-dlp

In [ ]:
!sudo apt install ffmpeg

## ⏬ Download


*   --write-subs --sub-lang en: chỉ tải sub english không tải file.
*   --all-subs: tải file và tất cả subtitle.
* --skip-download: bỏ qua không tải files
* -o: đặt tên file
* -P: path lưu
* -F: xem tất cả định dạng
* -f tải định dạng đã chọn
* !yt-dlp: tự chọn định dạng tốt nhất
* -v: để xem log
* --get-urlurl: tải .m3u8



In [ ]:
!yt-dlp --all-subs "https://wetv.vip/vi/play/87t5rpu3kbh1qm8/" -o "Xin_Hay_Yeu_Em_E7" -P "Dame na Watashi ni Koishite Kudasai 2016"